<a href="https://colab.research.google.com/github/jlopetegui98/Creation-of-a-synthetic-dataset-for-French-NER-in-clinical-trial-texts/blob/main/TranslationModel/evaluate_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# uncomment the following line to run in colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
!pip install sacrebleu

In [19]:
# uncomment if using colab
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U datasets
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U evaluate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [20]:
import evaluate
import transformers
import os
import json

In [21]:
# paths to the data
# root_path = '..' # comment if using colab
root_path = './drive/MyDrive/HandsOn-NLP/' # uncomment if using colab
data_path = f'{root_path}/data'
french_texts_path = f'{data_path}/chia_criteria_french'
chat_gpt_translations_path = f'{data_path}/chia_criteria_french_chat_gpt'
t5_translations_path = f'{data_path}/chia_criteria_french_t5'

In [22]:
# load files used for evaluation
files = os.listdir(chat_gpt_translations_path)

In [24]:
all_sentences = []
all_sentences_t5 = []
all_sentences_gpt = []

for file in files:
    with open(f'{french_texts_path}/{file}', 'r') as f:
        sentences = [line for line in f.readlines() if (line.strip() != '' and line.strip() != '\n' and len(line.split()) < 500) and len(line.split()) >= 1]
    with open(f'{t5_translations_path}/{file}', 'r') as f:
        sentences_t5 = [line for line in f.readlines() if (line.strip() != '' and line.strip() != '\n' and len(line.split()) < 500) and len(line.split()) >= 1]
    with open(f'{chat_gpt_translations_path}/{file}', 'r') as f:
        sentences_gpt = [line for line in f.readlines() if (line.strip() != '' and line.strip() != '\n' and len(line.split()) < 500) and len(line.split()) >= 1]

    if len(sentences) != len(sentences_t5):
        continue
    if len(sentences) != len(sentences_gpt):
        continue
    if len(sentences_t5) != len(sentences_gpt):
        continue
    for i in range(len(sentences)):
        if len(sentences[i].split()) < 2 or len(sentences_t5[i].split()) < 2 or len(sentences_gpt[i].split()) < 2:
          continue

        all_sentences.append(sentences[i])
        all_sentences_t5.append(sentences_t5[i])
        all_sentences_gpt.append(sentences_gpt[i])

In [25]:
len(all_sentences), len(all_sentences_t5), len(all_sentences_gpt)

(666, 666, 666)

**Evaluate perplexity for the model used in translations: Helsinki-NLP/opus-mt-en-fr**

In [ ]:
perplexity = evaluate.load("perplexity", module_type="metric")

results = perplexity.compute(model_id='Helsinki-NLP/opus-mt-en-fr',
                             add_start_token=False,
                             predictions=all_sentences)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


  0%|          | 0/36 [00:00<?, ?it/s]

In [ ]:
with open(f'{data_path}/perplexity_results.json', 'w') as f:
    json.dump(results, f)

**Evaluate perplexity for t5 model**

In [ ]:
# results_t5 = perplexity.compute(model_id='google-t5/t5-small',
#                              add_start_token=False,
#                              predictions=all_sentences_t5)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

ValueError: Unrecognized configuration class <class 'transformers.models.t5.configuration_t5.T5Config'> for this kind of AutoModel: AutoModelForCausalLM.
Model type should be one of BartConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BioGptConfig, BlenderbotConfig, BlenderbotSmallConfig, BloomConfig, CamembertConfig, LlamaConfig, CodeGenConfig, CpmAntConfig, CTRLConfig, Data2VecTextConfig, ElectraConfig, ErnieConfig, FalconConfig, FuyuConfig, GemmaConfig, GitConfig, GPT2Config, GPT2Config, GPTBigCodeConfig, GPTNeoConfig, GPTNeoXConfig, GPTNeoXJapaneseConfig, GPTJConfig, LlamaConfig, MarianConfig, MBartConfig, MegaConfig, MegatronBertConfig, MistralConfig, MixtralConfig, MptConfig, MusicgenConfig, MvpConfig, OpenLlamaConfig, OpenAIGPTConfig, OPTConfig, PegasusConfig, PersimmonConfig, PhiConfig, PLBartConfig, ProphetNetConfig, QDQBertConfig, Qwen2Config, ReformerConfig, RemBertConfig, RobertaConfig, RobertaPreLayerNormConfig, RoCBertConfig, RoFormerConfig, 

In [ ]:
with open(f'{data_path}/perplexity_results_gpt2.json', 'w') as f:
    json.dump(results_gpt2, f)

**BLEU evaluation**

We are going to use some translations from *chatGPT* as reference to get the BLEU score for the translations used in our task.

In [ ]:
# # get chat_gpt translations
# files_gpt = os.listdir(chat_gpt_translations_path)

# all_sentences_gpt = []
# all_sentences_to_evaluate = []
# for file in files_gpt:
#     with open(f'{chat_gpt_translations_path}/{file}', 'r') as f:
#         all_sentences_gpt.extend([line for line in f.readlines() if (line.strip() != '' and line.strip() != '\n' and len(line.split()) < 500) and len(line.split()) >= 3])
#     with open(f'{french_texts_path}/{file}', 'r') as f:
#         all_sentences_to_evaluate.extend([line for line in f.readlines() if (line.strip() != '' and line.strip() != '\n' and len(line.split()) < 500) and len(line.split()) >= 3])

In [26]:
bleu = evaluate.load("bleu")
results_bleu_hlk = bleu.compute(predictions=all_sentences, references=all_sentences_gpt)

In [27]:
results_bleu_hlk

{'bleu': 0.45827523271413784,
 'precisions': [0.7601043024771839,
  0.5988178398710371,
  0.49600648223173976,
  0.4120075046904315],
 'brevity_penalty': 0.8298245643303954,
 'length_ratio': 0.8427859014453554,
 'translation_length': 9971,
 'reference_length': 11831}

In [28]:
# save results
with open(f'{data_path}/bleu_results.json', 'w') as f:
    json.dump(results_bleu_hlk, f)

In [29]:
# check bleu for t5
results_bleu_t5 = bleu.compute(predictions=all_sentences_t5, references=all_sentences_gpt)

In [30]:
results_bleu_t5

{'bleu': 0.37254559408621185,
 'precisions': [0.7077648989639963,
  0.5148078828580865,
  0.3983604609718427,
  0.3118168959753118],
 'brevity_penalty': 0.8077034015292806,
 'length_ratio': 0.8240216380694785,
 'translation_length': 9749,
 'reference_length': 11831}

In [31]:
# save results
with open(f'{data_path}/bleu_results_t5.json', 'w') as f:
    json.dump(results_bleu_t5, f)

**TER(Translation Edit Rate) evaluation**

We will use the same approach as for BLEU score case. (using *chatGPT* translations as reference)

In [32]:
ter = evaluate.load("ter")
results_ter_hlk = ter.compute(predictions=all_sentences, references=all_sentences_gpt)

In [33]:
results_ter_hlk

{'score': 52.602065912444665, 'num_edits': 5347, 'ref_length': 10165.0}

In [34]:
# save results
with open(f'{data_path}/ter_results.json', 'w') as f:
    json.dump(results_ter_hlk, f)

In [35]:
results_ter_t5 = ter.compute(predictions=all_sentences_t5, references=all_sentences_gpt)

In [36]:
results_ter_t5

{'score': 60.02951303492375, 'num_edits': 6102, 'ref_length': 10165.0}

In [37]:
# save results
with open(f'{data_path}/ter_results_t5.json', 'w') as f:
    json.dump(results_ter_t5, f)

**METEOR (Metric for Evaluation of Translation with Explicit ORdering)**

We will use the same approach as for BLEU score case. (using *chatGPT* translations as reference)

In [39]:
meteor = evaluate.load("meteor")
results_meteor = meteor.compute(predictions=all_sentences, references=all_sentences_gpt)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [40]:
results_meteor

{'meteor': 0.6137599707346333}

In [41]:
# save results
with open(f'{data_path}/meteor_results.json', 'w') as f:
    json.dump(results_meteor, f)

In [42]:
results_meteor_t5 = meteor.compute(predictions=all_sentences_t5, references=all_sentences_gpt)

In [43]:
results_meteor_t5

{'meteor': 0.5500517585237066}

In [44]:
# save results
with open(f'{data_path}/meteor_results_t5.json', 'w') as f:
    json.dump(results_meteor_t5, f)

In [60]:
# show the results
import pandas as pd

results = pd.DataFrame(index=['Helsinki-NLP/opus-mt-en-fr', 't5'] ,columns = ['bleu', 'ter', 'meteor'])
results

,bleu,ter,meteor
Helsinki-NLP/opus-mt-en-fr,NaN,NaN,NaN
t5,NaN,NaN,NaN


In [61]:
# load results for bleu
with open(f'{data_path}/bleu_results.json', 'r') as f:
    results_bleu_hlk = json.load(f)
with open(f'{data_path}/bleu_results_t5.json', 'r') as f:
    results_bleu_t5 = json.load(f)
results['bleu'] = [results_bleu_hlk['bleu'], results_bleu_t5['bleu']]

In [62]:
# load results for ter
with open(f'{data_path}/ter_results.json', 'r') as f:
    results_ter_hlk = json.load(f)
with open(f'{data_path}/ter_results_t5.json', 'r') as f:
    results_ter_t5 = json.load(f)
results['ter'] = [results_ter_hlk['score'], results_ter_t5['score']]
# results['ter_num_edits'] = [results_ter_hlk['num_edits'], results_ter_t5['num_edits']]

In [63]:
# load results for meteor
with open(f'{data_path}/meteor_results.json', 'r') as f:
    results_meteor = json.load(f)
with open(f'{data_path}/meteor_results_t5.json', 'r') as f:
    results_meteor_t5 = json.load(f)
results['meteor'] = [results_meteor['meteor'], results_meteor_t5['meteor']]

In [64]:
results.T

,Helsinki-NLP/opus-mt-en-fr,t5
bleu,0.458275,0.372546
ter,52.602066,60.029513
meteor,0.613760,0.550052
